In [1]:
import pandas as pd

In [2]:
annot_tweets = pd.read_parquet('annotated_tweets.parquet')
annot_tweets.shape

(157, 24)

In [3]:
annot_tweets.head(10)

,id,tweet,highlight,not_good,polarized,tweet_bias,toxic_language,toxic_language_txt,hate_speech,hate_speech_txt,...,dehumanization,dehumanization_txt,crime_imputation,crime_imputation_txt,divisive_language,divisive_language_txt,strong_bias,strong_bias_txt,other,other_txt
0,1,"“PERDEU, MANÉ”! Foi isso mesmo que o ministro ...",1,0,1,1,0,,0,,...,0,,0,,0,,0,,0,
1,2,É fundamental q toda pessoa q está sendo imped...,1,0,1,-1,0,,0,,...,0,,0,,1,diferente do lado de lá,0,,0,
2,3,🚨 URGENTE! O diretor bolsonarista da Polícia R...,1,0,1,-1,0,,0,,...,0,,1,ISSO É UM GOLPE DE ESTADO! #Eleicoes2022,0,,0,,0,
3,4,"🚨URGENTE: Quer dizer que os ""patriotas"" iriam ...",1,0,1,1,0,,0,,...,1,infiltrados; terroristas,1,"infiltrados, terroristas, antifas, black block...",0,,0,,0,
4,5,Se você roubasse uma eleição em favor de um cr...,1,0,1,1,0,,0,,...,0,,1,Se você roubasse uma eleição em favor de um cr...,0,,0,,0,
5,6,"Renata Vasconcellos: ""Em época de pandemia, o ...",1,0,0,0,0,,0,,...,0,,0,,0,,0,,0,
6,7,Vamos simplificar... Poste aqui nesse tweet TO...,1,0,1,-1,0,,0,,...,0,,0,,0,,0,,1,Bora criar uma listinha pra facilitar os traba...
7,8,O mínimo que deve acontecer depois do golpismo...,1,0,1,-1,0,,0,,...,0,,1,Responsabilização da Jovem Pan,0,,0,,0,
8,9,🚨URGENTE!🚨Cacique Xavante Tserere ACABA DE SER...,0,0,1,1,0,,0,,...,0,,0,,0,,0,,1,Uma afronta direta ao presidente e uma escalad...
9,10,"No Nordeste, a Polícia Rodoviária Federal desc...",1,0,0,0,0,,0,,...,0,,0,,0,,0,,0,


In [8]:
left_tweets = annot_tweets[annot_tweets['tweet_bias']==-1].sample(45)
left_tweets.shape

(45, 24)

In [9]:
right_tweets = annot_tweets[annot_tweets['tweet_bias']==1].sample(45)
right_tweets.shape

(45, 24)

In [11]:
random_sample = pd.concat([left_tweets, right_tweets]).sample(frac=1)
random_sample.shape

(90, 24)

In [12]:
random_sample.head()

,id,tweet,highlight,not_good,polarized,tweet_bias,toxic_language,toxic_language_txt,hate_speech,hate_speech_txt,...,dehumanization,dehumanization_txt,crime_imputation,crime_imputation_txt,divisive_language,divisive_language_txt,strong_bias,strong_bias_txt,other,other_txt
67,68,"🚨 Em Niterói, uma das poucas cidades onde Lula...",0,0,1,-1,0,,0,,...,0,,1,a Polícia Rodoviária Federal está tentando imp...,0,,0,,0,
72,73,Vamos desenhar o ministro Barroso em três fase...,0,0,1,1,1,Esse é o estado democrático de direito da repú...,0,,...,0,,0,,0,,0,,1,
115,116,A Força Nacional vai partir pra cima dos manif...,0,0,1,1,0,,0,,...,0,,0,,1,Lula massacrando o povo de bem,0,,1,Lula massacrando o povo de bem
30,31,"É preciso esclarecer, o relatório do PL, basea...",1,0,1,1,0,,0,,...,0,,0,,0,,0,,1,. #BolsonaroReeleito com larga vantagem!
62,63,O João Salas transmitiu toda a movimentação de...,0,0,1,1,0,,0,,...,0,,0,,0,,0,,0,


In [13]:
def define_bias(bias_val):
    if bias_val == -1:
        return 'L'
    elif bias_val == 0:
        return 'C'
    elif bias_val == 1:
        return 'R'
    else:
        return 'U'

In [20]:
tweets = []
for idx, row in random_sample.iterrows():
    id = define_bias(row['tweet_bias'])+str(idx).zfill(3)
    tweets.append({
        'tweet_id': id,
        'tweet_text': row['tweet'],
        'bias': define_bias(row['tweet_bias'])
    })

In [21]:
indexed_tweets = pd.DataFrame.from_records(tweets)

In [22]:
indexed_tweets.head()

,tweet_id,tweet_text,bias
0,L067,"🚨 Em Niterói, uma das poucas cidades onde Lula...",L
1,R072,Vamos desenhar o ministro Barroso em três fase...,R
2,R115,A Força Nacional vai partir pra cima dos manif...,R
3,R030,"É preciso esclarecer, o relatório do PL, basea...",R
4,R062,O João Salas transmitiu toda a movimentação de...,R


In [31]:
human = [
    indexed_tweets[indexed_tweets['bias']=='L'][1:15],
    indexed_tweets[indexed_tweets['bias']=='R'][1:15]
]
machine = [
    indexed_tweets[indexed_tweets['bias']=='L'][16:30],
    indexed_tweets[indexed_tweets['bias']=='R'][16:30]
]
placebo = [
    indexed_tweets[indexed_tweets['bias']=='L'][31:45],
    indexed_tweets[indexed_tweets['bias']=='R'][31:45]
]

human_tts = pd.concat(human)
machine_tts = pd.concat(machine)
placebo_tts = pd.concat(placebo)

In [35]:
human_tts['rephrase_id'] = human_tts['tweet_id'].apply(lambda x: 'H'+x)
machine_tts['rephrase_id'] = machine_tts['tweet_id'].apply(lambda x: 'M'+x)
placebo_tts['rephrase_id'] = placebo_tts['tweet_id'].apply(lambda x: 'P'+x)

In [36]:
machine_tts.shape

(28, 4)

In [37]:
human_tts.to_parquet('human_groups_tweets.parquet')
machine_tts.to_parquet('machine_groups_tweets.parquet')
placebo_tts.to_parquet('placebo_groups_tweets.parquet')